In [1]:
# Imports

import random
import solution

## Wordle Solver

Wordle is a word guessing game where the user must guess a 5 letter word within 6 guesses by guessing a single word at a time.  The player will then receive feedback on whether or not each letter in the word is:

1. Not in the word at all. (False prediction)
2. In the word, but not in the given position. (False position)
3. In the word, at the guessed position. (True prediction)

### Task 1: An English Wordle solver.

#### Instructions

There are many strategies to select words that will minimize the number of guesses that a player needs to make: choose a word with as many unguessed letters as possible, to obtain information about each letter; maximize choices to consist of the most common letters; maximize choices to consist of letters in their most frequent positions, etc.

Your task is this.  Given a list of acceptable words in "English_words.txt" (we're changing things up a bit, and giving you words of length *6*, instead of length 5), decide upon a strategy to choose the most likely solution. You are free to use any strategy you want.

You will likely want to write functions that can:

1. read a list of words into a data structure
2. perform some statistical analysis of the data
3. update the list, given the response of the Wordle game.  All you need to know about the algorithm is that it will accept a suggestion for the word, and will return a list that will correspond to one of the three options cited above.  For example, if the solution is "parse", and you provide "morph", the algorithm will return. [-1,-1,1,0,-1] (ie, the character at position "2" is true, and the one at position "3" is a false position; everything else is a false prediction). You will likely need to make different decisions, depending on the results.  You are *strongly* encouraged to test this functionality thoroughly.
4. If you have guessed the word (ie, the return of the algorithm is [1,1,1,1,1]), then you are done, and you have won the game!  Good job!  If not, then you will need to recalculate the most likely word, given your strategy.

You are free to tackle this problem however you see fit, but here are some strategies that might work:
    * Find common letter distribution patterns (do you want to use extra data?)
    * Find common positional letter distributions
    * Keep track of valid words, and score them according to your statistics, always choosing the most likely one.
    * Choose the one that gives you the most information (not necessarily the same as the previous point).
    * Choose the one that eliminates the most remaining letters.
    * Pick a starting word at random
    * Pick a starting word that is filled with common letters
    * Pick a starting word that has all different letters
    * etc.  Have fun!

In [2]:
# Change file path as appropriate

file_path = ""

#### Evaluation

From the list of words we provided, we will have selected 1000 of them, and calculate the average number of guesses your system needs to correctly identify the word (unlike the true Wordle, we will not cap guesses at 6).  The team with the best average score will earn a bonus percentage point on the Capstone; the second-best team will earn a half-point. All of the words in the word list are relatively common (they appear at least twice in the Brown corpus). You will need to submit your code, as well as instructions on how to run it.  An example of the evaluation code is provided below.

In [3]:
def giveFeedback(gold, prediction):  # Provided
    '''
       Provided two lists, (the reason we use lists, 
       and not strings, is because of part 2 of this assignment)
       return a list that indicates whether a letter is in the
       right location (1), is in the word, 
       but not in the right location (0), 
       or is not in the word at all (-1)

       Params:

           gold - a list of strings corresponding to letters in the secret word
           prediction - a list of strings corresponding to letters in the guess

       Return values:

           feedback - a list of integers in the range [-1-1], of the same length as the secret word
    '''
    feedback = []
    if(len(gold) != len(prediction)):
        return feedback

    for i in range(len(gold)):
        if(gold[i] == prediction[i]):  #The easiest possibility: we got the prediction right
            feedback.append(1)
        elif(prediction[i] in gold):   #If the letter is somewhere in the gold word
            feedback.append(0)
        else:
            feedback.append(-1)        #If the predicted letter is nowhere in the gold word

    return feedback


In [4]:
giveFeedback(list("story"), list("stony"))

[1, 1, 1, -1, 1]

#### Basic logic

In the evaluation loop, we are going to be calling a `remove_words` function based on the feedback to filter the words to the ones that are possible given the most recent feedback. This generates a new "active" file (see the evaluation loop below) which serves as the input to our `make_guess` function.

In [5]:
def remove_words(file, guess, feedback):
    
    with open(file_path + file) as f:
        word_list=[line.strip() for line in f.readlines()]

    to_remove = []

    for i, num in enumerate(feedback):
        if num == 1:
            for word in word_list:
                if word[i] != guess[i]:
                    to_remove.append(word)

        if num == -1:
            for word in word_list:
                for char in word:
                    if char == guess[i]:
                        to_remove.append(word)
                        
        if num == 0:
            for word in word_list:
                if word[i] == guess[i]:
                    to_remove.append(word)
                if guess[i] not in word:
                    to_remove.append(word)

    for word in to_remove:
        if word in word_list:
            word_list.remove(word)

    with open(file_path + file, "w") as f:
        for word in word_list:
            f.write(word + "\n")

### How to run our code in the loop

Below is an example of how to run our code. For each word in the test set, make as many guesses as desired; keep track of the number of guesses as this is required as an argument to the `make_guess` function for the code to get the best results. After generating the guess, get the feedback using the `giveFeedback` function, and input this to the `remove_words` function along with the active list of words and the feedback.

In [6]:
english_list = solution.english_list
golds = random.sample(english_list, 1000)

with open(file_path + "active_list.txt", "w") as f:
    for word in english_list:
        f.write(word + "\n")

num_guesses = []

for gold in golds:
    with open(file_path + "active_list.txt", "w") as f:
        for word in english_list:
            f.write(word + "\n")
    guess_count = 0
    guess = ""
    while guess != gold:
        guess = solution.make_guess("active_list.txt", guess_count)
        guess_count += 1
        if guess == gold:
            num_guesses.append(guess_count)
            break
        feedback = giveFeedback(gold, guess)
        remove_words("active_list.txt", guess, feedback)

print(sum(num_guesses)/len(num_guesses))

3.21


The Pseudocode of the evaluation is as follows: 
    
AVERAGE_COUNT:=0

FOR: WORD IN GOLD WORD_LIST: <br>
     &ensp; COUNT:=0 <br>
     &ensp; FEEDBACK:=NONE <br>
     &ensp; WHILE: GUESS != WORD: <br>
         &ensp; &ensp; GUESS = YOUR_FUNCTION.MAKE_GUESS(FEEDBACK) <br>
         &ensp; &ensp; FEEDBACK = GIVE_FEEDBACK(WORD, GUESS) <br>
         &ensp; &ensp; COUNT += 1 <br>
     &ensp; AVERAGE_COUNT+=COUNT <br>
AVERAGE_COUNT /= WORD_LIST.SIZE


Your "make_guess" function should take in feedback in the form described above (possibly an empty list, for the first turn), and calculate the most likely word, given your calculations.  Note that each call to the function will re-instantiate your code, so you might want to keep track of the valid words somewhere - do not use global variables!



### Task 2: A Gitksan Wordle solver.

Gitksan is the language of the Gitxsan people of British Columbia.  It belongs to the Tsimshianic language family, and has far fewer speakers than English.  The traditional territories consist of 50,000 square km in the Skeena river watershed (pictured below).  The language is written using a modified version of the Latin alphabet, with both digraphs/trigraphs (two/three letters that represent one sound, like 'th' in English) and diacritics (accents or markings that change the meaning of the letter).  For example, "hl" is considered one letter, representing a single sound, as is <ins>k</ins>.  The letters used in Gitksan are as follow (sourced from : Gitksan, by Jason Brown, Henry Davis, Michael Schwan and Barbara Sennott).  Thanks to the Gitksan Lab in the UBC department of Linguistics for providing the dictionary, and being very enthusiastic about this coding problem.  
<img src="img/Skeena_river_basin_map.png" alt="Traditional Gitxsan territory" width="200"/>

p
b
d
t
k
kw
<ins>k</ins>
gy
gw
<ins>g</ins>
ts
j
p'
t'
ky'
kw'
<ins>k</ins>'
ts'
tl'
m
n
s
x
xw
<ins>x</ins>
h
hl
y
w
l
'm
'n
'l
'y
'w
i
ii
ee
a
aa
oo
u
uu

Now, you will need to add an additional step of breaking your words into characters, and recalculating your statistics based on those letters.  'laaxw' ("trout") is not [l, a, a, x, w], but rather, [l, aa, xw]. In the provided 'Gitksan5.txt' file, underlined characters are represented with a following underscore: _. You will have to determine your own way of segmenting the data to represent Gitksan letters.
    
There may be instances where you have to make a choice in how a word is segmented, and it may impact your results slightly.  It would take a certain level of fluency in Gitksan to always get this correct, but don't worry - consistency is more important than 100% accuracy.
    
    

#### Evaluation (code not provided)

Just as with the English solver, we will evaluate on a set of words, and the team with the lowest average number of guesses will win a bonus percentage point.  The team in second place will win a half a percentage point.  This applies to both parts - theoretically, one team could win 2 percentage points.

Since we only have limited data for Gitksan, we will consider one of the largest subsets within the data - words of 5 characters.  We are counting digraphs / trigraphs as 1 character, and the space is also 1 character.  There are ~200 5 "letter" words in the data set - we will evaluate on a random selection of 100 of them.

In [7]:
gitksan_list = solution.gitksan_list
gitksan_golds = random.sample(gitksan_list, len(gitksan_list)//2)

with open(file_path + "active_list.txt", "w") as f:
    for word in gitksan_list:
        f.write(solution.encode_word(word) + "\n")

num_guesses = []

for gold in gitksan_golds:
    with open(file_path + "active_list.txt", "w") as f:
        for word in gitksan_list:
            f.write(solution.encode_word(word) + "\n")
    guess_count = 0
    guess = ""
    gold = solution.encode_word(gold)
    while guess != gold:
        guess = solution.make_guess_gitksan("active_list.txt", guess_count)
        guess_count += 1
        if guess == gold:
            assert solution.encode_word(gold, "decode") in gitksan_list  # Ensuring we are guessing a real word
            num_guesses.append(guess_count)
        feedback = giveFeedback(gold, guess)
        remove_words("active_list.txt", guess, feedback)

print(sum(num_guesses)/len(num_guesses))

3.0869565217391304
